In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from copy import deepcopy
from scipy.sparse.csgraph import laplacian
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from invert import focuss, inverse_msp
from invert.cmaps import parula
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi128')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape
dist = cdist(pos, pos)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.0s remaining:    2.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.6s finished


# Get sample data

In [3]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=4, extents=(1, 40), duration_of_trial=1, target_snr=999)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
M = evoked.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 222.82it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00,  3.49it/s]


Using pyvista 3d backend.

Using control points [1.99677851e-09 4.18159124e-09 3.53635563e-08]


# Minimum Norm Estimate

In [ ]:
alpha = 0.001
# if n_chans>n_dipoles:
# D_MNE = np.linalg.inv(leadfield.T @ leadfield + alpha * np.identity(n_dipoles)) @ leadfield.T @ M
# else:
D_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans)) @ M


stc_hat = stc.copy()
stc_hat.data = D_MNE
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'MNE', 'title',
               font_size=14)

# Weighted Minimum Norm Estimate

In [4]:
alpha = 0.001
omega = np.diag(np.linalg.norm(leadfield, axis=0))
I_3 = np.identity(3)
W = omega # np.kron(omega, I_3)

D_WMNE = np.linalg.inv(W.T @ W) @ leadfield.T  @ np.linalg.inv(leadfield @ np.linalg.inv(W.T @ W) @ leadfield.T + alpha * np.identity(n_chans)) @ M

stc_hat = stc.copy()
stc_hat.data = D_WMNE
r = np.median([pearsonr(abs(s_pred), abs(s_true))[0] for s_pred, s_true in zip(stc_hat.data.T, stc.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'wMNE (r={r:.2f})', 'title',
               font_size=14)

Using control points [1.73250790e-09 2.43647303e-09 1.20457740e-08]


# MNE with FOCUSS (Focal underdetermined system solution)

In [ ]:
alpha = 0.001

D_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans)) @ M
D_FOCUSS = focuss(D_MNE, M, leadfield, alpha)



stc_hat = stc.copy()
stc_hat.data = D_FOCUSS
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'MNE FOCUSS', 'title',
            font_size=14)

# Contextual Minimum Norm

In [5]:
from invert import inverse_loreta, contextualize, contextualize_bd
import tensorflow as tf
stc_instant = inverse_loreta(M, leadfield, fwd)
stc_cmne = contextualize_bd(stc_instant, fwd, num_epochs=20)
from scipy.stats import pearsonr

stc = sim.source_data[0]


stc_hat = stc.copy()
stc_hat.data = stc_cmne
r = np.median([pearsonr(abs(s_pred), abs(s_true))[0] for s_pred, s_true in zip(stc_hat.data.T, stc.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'stc_cmne (r={r:.2f})', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               723456    
                                                                 
 dense (Dense)               (None, 1284)              165636    
                                                                 
Total params: 889,092
Trainable params: 889,092
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
49/49 [==============================] - 6s 52ms/step - loss: 0.0066 - cosine_similarity: -0.7981 - val_loss: 0.0053 - val_cosine_similarity: -0.8823
Epoch 2/20
49/49 [==============================] - 1s 28ms/step - loss: 0.0025 - cosine_similarity: -0.9479 - val_loss: 0.0049 - val_cosine_similarity: -0.8908
Epoch 3/20
49/49 [==============================] - 1s 27ms/step - loss: 0.0024 - cosi

# dSPM

In [ ]:
alpha = 0.001

noise_cov = np.identity(n_chans) + np.random.rand(n_chans, n_chans)*0.1
source_cov = np.identity(n_dipoles)

M_norm = (1/np.sqrt(noise_cov)) @ M
G_norm = (1/np.sqrt(noise_cov)) @ leadfield

K = source_cov @ G_norm.T @ np.linalg.inv(G_norm @ source_cov @ G_norm.T + alpha**2 * np.identity(n_chans))
W_dSPM = np.diag(np.sqrt(1/np.diagonal(K @ noise_cov @ K.T)))
K_dSPM = W_dSPM @ K
D_dSPM = K_dSPM @ M_norm

# rectify & normalize
D_dSPM_norm = np.stack([(x-abs(x).mean()) / abs(x).std() for x in D_dSPM.T], axis=1)


stc_hat = stc.copy()
stc_hat.data = D_dSPM_norm
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'dSPM', 'title',
               font_size=14)

# LORETA

In [6]:
alpha = 0.001
adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
B = np.diag(np.linalg.norm(leadfield, axis=0))
laplace_operator = laplacian(adjacency)
D_LOR = np.linalg.inv(leadfield.T @ leadfield + alpha * B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T @ M
# D_LOR = np.linalg.inv(B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T @ (leadfield @ np.linalg.inv(B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T + alpha * np.identity(n_chans) ) @ M

stc_hat = stc.copy()
stc_hat.data = D_LOR
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'LORETA (r={r:.2f})', 'title',
            font_size=14)

-- number of adjacent vertices : 1284
Using control points [3.25766750e-09 4.39960669e-09 1.96530940e-08]


# sLORETA

In [7]:
alpha = 0.001
K_MNE = leadfield.T @ np.linalg.inv(leadfield @ leadfield.T + alpha * np.identity(n_chans))
D_MNE = K_MNE @ M
W_diag = 1 / np.diag(K_MNE @ leadfield)

W_slor = np.diag(W_diag)

W_slor = np.sqrt(W_slor)

K_slor = W_slor @ K_MNE
D_SLOR = K_slor @ M
D_SLOR[np.isnan(D_SLOR)] = 0

stc_hat = stc.copy()
stc_hat.data = D_SLOR
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'sLORETA (r={r:.2f})', 'title',
            font_size=14)

Using control points [8.67399755e-09 1.15403658e-08 4.29932164e-08]


# eLORETA

In [ ]:
from invert import calc_eloreta_D
stop_crit = 0.005
alpha = 0.001

D, C = calc_eloreta_D(leadfield, alpha, stop_crit=stop_crit)
K_elor = np.linalg.inv(D) @ leadfield.T @ np.linalg.inv( leadfield @ np.linalg.inv(D) @ leadfield.T + alpha * np.identity(n_chans) )
D_ELOR = K_elor @ M

stc_hat = stc.copy()
stc_hat.data = D_ELOR
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'eLORETA', 'title',
            font_size=14)


# LAURA

In [8]:
alpha = 200
drop_off = 2
d = cdist(pos, pos)
# Get the adjacency matrix of the source spaces
adj = mne.spatial_src_adjacency(fwd["src"], verbose=0).toarray()
for i in range(d.shape[0]):
    # find dipoles that are no neighbor to dipole i
    non_neighbors = np.where(~adj.astype(bool)[i, :])[0]
    # append dipole itself
    non_neighbors = np.append(non_neighbors, i)
    # set non-neighbors to zero
    d[i, non_neighbors] = 0
A = -d**-drop_off
A[np.isinf(A)] = 0
W = np.identity(A.shape[0])
M_j = W @ A

# Source Space metric
W_j = np.linalg.inv(M_j.T @ M_j)
W_j_inv = np.linalg.inv(W_j)

W_d = np.linalg.inv(np.identity(n_chans))
noise_term = (alpha**2) * np.linalg.inv(W_d)
G = W_j_inv @ leadfield.T @ np.linalg.inv(leadfield @ W_j_inv @ leadfield.T + noise_term)
D_LAURA = G @ M

stc_hat.data = D_LAURA
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'LAURA (r={r:.2f})', 'title',
            font_size=14)

C:\Users\Lukas\AppData\Local\Temp/ipykernel_936/478972351.py:13: RuntimeWarning: divide by zero encountered in power
  A = -d**-drop_off


Using control points [9.67561006e-10 1.30718866e-09 4.98922947e-09]


# VARETA

In [ ]:
# 1) J & JVareta given; find A
# 2) A given, find J
# 3) repeat until convergence

In [ ]:
 
# alpha = 0.001
# adjacency = mne.spatial_src_adjacency(fwd['src'], verbose=0).toarray()
# B = np.diag(np.linalg.norm(leadfield, axis=0))
# L = laplacian(adjacency)  # non-singular univariate discrete laplacian -> is that correct?
# D_LOR = np.linalg.inv(leadfield.T @ leadfield + alpha * B @ L.T @ L @ B) @ leadfield.T @ M
# D_Last = deepcopy(D_LOR)[:, 0][:, np.newaxis]
# W = np.diag(np.linalg.norm(leadfield, axis=0))  # WMNE weight matrix (depth weighting)
# tau = 1  # controls smoothness
# alpha_2 = 1  # controls importance of grid point
# # for t in range(M.shape[1]):
# t = 0
# A = np.identity(n_dipoles)  # in paper: large lambda
    
# # for _ in range(10):
# term_1 = np.linalg.norm( M[:, t] - leadfield @ D_Last )
# term_2 = np.linalg.norm( A@L * W * D_Last )
# term_3 = tau**2 * np.linalg.norm( L * np.diag(np.log(np.diagonal(A))) - alpha_2 )


# # D_VAR = term_1 + term_2 + term_3

# from scipy.optimize import minimize
# def find_lambda(A, m, leadfield, D_Last, L, W, tau, alpha_2):
#     A = A.reshape(leadfield.shape[1], leadfield.shape[1])
#     term_1 = np.linalg.norm( M[:, t] - leadfield @ D_Last )
#     term_2 = np.linalg.norm( A@L * W * D_Last )
#     term_3 = tau**2 * np.linalg.norm( L * np.diag(np.log(np.diagonal(A))) - alpha_2 )
#     return term_1 + term_2 + term_3
# # find_lambda(A, M[:, t], leadfield, D_Last, L, W, tau, alpha_2)
# minimize(find_lambda, A.flatten(), args=(M[:, t], leadfield, D_Last, L, W, tau, alpha_2), method='L-BFGS-B')

# S-MAP

In [ ]:
alpha = 0.001
adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
B = np.diag(np.linalg.norm(leadfield, axis=0))
laplace_operator = laplacian(adjacency)
D_LOR = np.linalg.inv(leadfield.T @ leadfield + alpha * B @ laplace_operator.T @ laplace_operator @ B) @ leadfield.T @ M

gradient = np.gradient(B)[0] #np.gradient(B)[0]
D_SMAP = np.linalg.inv(leadfield.T @ leadfield + alpha * gradient.T @ gradient) @ leadfield.T @ M
# D_SMAP = np.linalg.inv(gradient.T @ gradient) @ leadfield.T @ np.linalg.inv(leadfield @ np.linalg.inv( gradient.T @ gradient ) @ leadfield.T + alpha * np.identity(n_chans)) @ M

stc_hat = stc.copy()
stc_hat.data = D_SMAP
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'S-MAP', 'title',
            font_size=14)

# ESINET

In [9]:
from esinet import Net
import tensorflow as tf
sim_train = Simulation(fwd, info, settings=dict(duration_of_trial=0)).simulate(5000)
net = Net(fwd, n_lstm_layers=0, activation_function='tanh').fit(sim_train)
D_ESINET = net.predict(sim.eeg_data[0])[0].data
from scipy.stats import pearsonr
stc_hat = stc.copy()
stc_hat.data = D_ESINET
r = np.mean([pearsonr(a, b)[0] for a, b in zip(stc_hat.data.T, stc.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'ESINET (r={r:.2f})', 'title',
            font_size=14)


D_cESINET = contextualize_bd(D_ESINET, fwd)
stc_hat.data = D_cESINET
brain = stc_hat.plot(**pp)
r = np.mean([pearsonr(a, b)[0] for a, b in zip(stc_hat.data.T, stc.data.T)])
brain.add_text(0.1, 0.9, f'cESINET, (r={r:.2f})', 'title',
            font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 5000/5000 [00:00<00:00, 20093.21it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 5000/5000 [00:10<00:00, 459.08it/s]


preprocess data
Model: "Dense-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 FC_0 (TimeDistributed)      (None, None, 100)         12900     
                                                                 
 Drop_0 (Dropout)            (None, None, 100)         0         
                                                                 
 FC_Out (TimeDistributed)    (None, None, 1284)        129684    
                                                                 
Total params: 142,584
Trainable params: 142,584
Non-trainable params: 0
_________________________________________________________________
fit model
Epoch 1/50
562/562 [==============================] - 5s 8ms/step - loss: -0.2833 - mae: 0.1418 - val_loss: -0.3637 - val_mae: 0.1279
Epoch 2/50
562/562 [==============================] - 5s 8ms/step - loss: -0.3633 - mae: 0.1376 - val_loss: -0.3896 - val_mae: 0.1313
Epoch 3/50
562/562 

c:\Users\Lukas\Envs\esienv\lib\site-packages\scipy\optimize\_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


Using control points [2.19824804e-09 3.17651904e-09 1.09103829e-08]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               723456    
                                                                 
 dense_3 (Dense)             (None, 1284)              165636    
                                                                 
Total params: 889,092
Trainable params: 889,092
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
49/49 [==============================] - 4s 42ms/step - loss: 0.0088 - cosine_similarity: -0.8252 - val_loss: 0.0057 - val_cosine_similarity: -0.9120
Epoch 2/100
49/49 [==============================] - 1s 27ms/step - loss: 0.0031 - cosine_similarity: -0.9585 - val_loss: 0.0052 - val_cosine_similarity: -0.9215
Epoch 3/100
49/49 [==============================] -

# Backus-Gilbert

In [ ]:
# Calculate distance matrix
dist = cdist(pos, pos)

print("Get W_BG")
W_BG = []
for i in range(n_dipoles):
    W_gamma_BG = np.diag(dist[i, :])
    W_BG.append(W_gamma_BG)

print("Get C")
C = []
for i in range(n_dipoles):
    C_gamma = leadfield @ W_BG[i] @ leadfield.T
    C.append(C_gamma)

print("Get F")
F = leadfield @ leadfield.T

print("Get E")
E = []
for i in range(n_dipoles):
    E_gamma = C[i] + F
    E.append(E_gamma)

print("Get L")
L = leadfield @ np.ones((n_dipoles, 1))

print("Get T")
T = []
for i in range(n_dipoles):
    E_gamma_pinv = np.linalg.pinv(E[i])
    T_gamma = (E_gamma_pinv @ L) / (L.T @ E_gamma_pinv @ L)
    T.append(T_gamma)

T_final = np.stack(T, axis=0)[:, :, 0]
D_BG = T_final @ M

stc_hat = stc.copy()
stc_hat.data = D_BG
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'Backus-Gilbert', 'title',
            font_size=14)

# Multiple Sparse Priors

In [11]:
# D_MSP = inverse_msp(evoked, fwd)

# stc_hat = stc.copy()
# stc_hat.data = D_MSP
# r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
# brain = stc_hat.plot(**pp)
# brain.add_text(0.1, 0.9, f'MSP (r={r:.2f})', 'title',
#             font_size=14)

D_bLORETA = inverse_msp(evoked, fwd, inversion_type="LORETA")

stc_hat = stc.copy()
stc_hat.data = D_bLORETA
r = np.median([pearsonr(a, b)[0] for a, b in zip(stc.data.T, stc_hat.data.T)])
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, f'Bayesian LORETA (r={r:.2f})', 'title',
            font_size=14)

Using 4 temporal mode(s)
ReML Iteration 0: 550.9152419787845
ReML Iteration 1: 134.68116963759292
ReML Iteration 2: 13.686518523341359
ReML Iteration 3: -43.196883857895656
Free-energy:  [[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
final h:  [  5.53019207 -22.03765817  -6.12698377]


c:\Users\Lukas\Documents\projects\invert\dev\..\invert\multiple_sparse_priors.py:896: RuntimeWarning: divide by zero encountered in true_divide
  Fc = Ft/2 + np.e*hP*np.e/2 + np.log(np.linalg.det( Ph/hP )) / 2
c:\Users\Lukas\Envs\esienv\lib\site-packages\numpy\linalg\linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


Using control points [2.63513856e-09 3.62934876e-09 1.29508499e-08]


Using control points [0.14648978 0.25055134 0.86284206]
Using control points [5.18381449e-10 2.04518638e-09 1.28947520e-08]
Using control points [0.19587537 0.34272974 0.94306367]
Using control points [0.14648978 0.25055134 0.86284206]
Using control points [0.14648978 0.25055134 0.86284206]
Using control points [1.10978955e-09 1.44626485e-09 5.22418641e-09]
Using control points [0.16640587 0.3928526  0.94690733]
Using control points [5.08090701e-09 9.01199477e-09 6.53284263e-08]
Using control points [0.16640587 0.3928526  0.94690733]
Using control points [0.03275672 0.10689903 0.95311963]
Using control points [4.39778270e-09 5.28808742e-09 1.53540128e-08]
Using control points [5.73386377e-10 1.96194089e-09 1.33111727e-08]
Using control points [0.14548956 0.32341721 0.83910691]
Using control points [5.73386377e-10 1.96194089e-09 1.33111727e-08]
Using control points [0.07228274 0.1141501  0.42193883]
Using control points [1.32864333e-09 1.54653927e-09 5.00582914e-09]
Using control points

  File "c:\Users\Lukas\Envs\esienv\lib\site-packages\mne\viz\_brain\_brain.py", line 1348, in _on_button_release
    self.picked_renderer = self.plotter.iren.FindPokedRenderer(x, y)
AttributeError: 'RenderWindowInteractor' object has no attribute 'FindPokedRenderer'


Using control points [3.05285365e-09 4.77026398e-09 7.46999654e-09]
Using control points [3.05285365e-09 4.77026398e-09 7.46999654e-09]



# LUCAS

In [ ]:
stc_hat.data = np.mean([D_MNE , D_WMNE, D_LOR, D_SLOR, D_ELOR, D_LAURA, D_SMAP, D_ESINET, D_BG, D_MSP], axis=0)
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'LUCAS', 'title',
            font_size=14)

stc_hat.data = contextualize(stc_hat.data, fwd)
brain = stc_hat.plot(**pp)
brain.add_text(0.1, 0.9, 'cLUCAS', 'title',
            font_size=14)

# Saved for later

In [ ]:
from esinet.util import get_eeg_from_source
evoked_hat = get_eeg_from_source(stc_hat, fwd, info, tmin=stc.tmin)
evoked_hat.plot_topomap()

In [ ]:
evoked.plot_topomap()